<a href="https://colab.research.google.com/github/Chrissek/dw_matrix_car/blob/master/day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car" 

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [0]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [0]:
xgb_params = {
    'max_depth':5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[22:06:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:06:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:07:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [0]:
m =xgb.XGBRFRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp, feature_names = cat_feats)

[22:09:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.0445 ± 0.0005,param_napęd__cat
0.0284 ± 0.0009,param_rok-produkcji__cat
0.0269 ± 0.0005,param_skrzynia-biegów__cat
0.0258 ± 0.0007,param_faktura-vat__cat
0.0135 ± 0.0004,param_moc__cat
0.0073 ± 0.0003,param_stan__cat
0.0064 ± 0.0002,feature_kamera-cofania__cat
0.0041 ± 0.0003,param_pojemność-skokowa__cat
0.0032 ± 0.0000,feature_łopatki-zmiany-biegów__cat
0.0016 ± 0.0000,feature_światła-led__cat


In [0]:
'param_napęd__cat',
'param_rok-produkcji__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_stan__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'seller_type__cat',
'feature_wspomaganie-kierownicy__cat',
'seller_name__cat',
'feature_elektrycznie-ustawiane-fotele__cat',
'param_wersja__cat',
'feature_system-start-stop__cat',
'feature_bluetooth__cat',
'param_typ__cat',
'feature_czujniki-parkowania-przednie__cat'

In [19]:
len(cat_feats)

151

In [0]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_stan__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'seller_type__cat',
'feature_wspomaganie-kierownicy__cat',
'seller_name__cat',
'feature_elektrycznie-ustawiane-fotele__cat',
'param_wersja__cat',
'feature_system-start-stop__cat',
'feature_bluetooth__cat',
'param_typ__cat',
'feature_czujniki-parkowania-przednie__cat']

In [36]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:38:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:38:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:38:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10148.23812468198, 65.15962449562089)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))


In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [39]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_stan__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'seller_type__cat',
'feature_wspomaganie-kierownicy__cat',
'seller_name__cat',
'feature_elektrycznie-ustawiane-fotele__cat',
'param_wersja__cat',
'feature_system-start-stop__cat',
'feature_bluetooth__cat',
'param_typ__cat',
'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:44:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10003.559859789288, 71.63849140290796)